# Import Libraries

In [ ]:
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

     |████████████████████████████████| 819 kB 4.2 MB/s 
     |████████████████████████████████| 65.1 MB 89 kB/s 
     |████████████████████████████████| 887 kB 55.7 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 121 kB 60.5 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 237 kB 76.6 MB/s 
     |████████████████████████████████| 4.0 MB 45.3 MB/s 
     |████████████████████████████████| 251 kB 73.6 MB/s 
     |████████████████████████████████| 462 kB 74.8 MB/s 
     |████████████████████████████████| 4.2 MB 40.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.28.1 which is incompatible.
pymc3 3.11.4 requires cachetools>=4.2.1, but you have cachetools 3.1.1 which is incompatible.
panel 0.1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff



SEED = 0
tf.random.set_seed(SEED)
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [ ]:
from tensorflow import keras
from keras.metrics import BinaryAccuracy, Precision, Recall
from keras import layers

from sklearn.model_selection import GroupShuffleSplit

# Load Data


In [ ]:
path = "/content/drive/MyDrive/Thesis/Datasets/Turbofan_Dataset/final_datasets_normalized/"

In [ ]:
# Load data and drop irrelevant columns

alice_set = pd.read_csv(path + "TRAINING_SET_1.csv")
bob_set = pd.read_csv(path + "TRAINING_SET_2.csv")

test_set = pd.read_csv(path + "TEST_SET_FULL.csv")


drop_cols = ["cycle","setting3","s1","s5","s10","s16","s18","s19","RUL"]
corr_cols = ["s11","s4","s15","s17","s2","s3","s8","s13","s9","s14","s12","s7","s20"]
feature_cols = ['cycle_norm', 'setting1', 'setting2', 's2', 's3', 's4', 's6', 's7',
       's8', 's9', 's11', 's12', 's13', 's14', 's15', 's17', 's20', 's21']
prediction_col = 'fail_30'

alice_set = alice_set.drop(drop_cols, axis=1)
bob_set = bob_set.drop(drop_cols, axis=1)

test_set = test_set.drop(drop_cols, axis=1)

In [ ]:
# Move cycle_norm column first for convenience

column_to_move = alice_set.pop("cycle_norm")
alice_set.insert(0, "cycle_norm", column_to_move)
column_to_move = bob_set.pop("cycle_norm")
bob_set.insert(0, "cycle_norm", column_to_move)

column_to_move = test_set.pop("cycle_norm")
test_set.insert(0, "cycle_norm", column_to_move)

In [ ]:
alice_set.shape

(9909, 20)

In [ ]:
bob_set.shape

(10722, 20)

In [ ]:
test_set.shape

(13096, 20)

# Sequence Generation

In [ ]:
# Receives single engine dataframe, window size and features -> sequences of length==window_size
def gen_train_data(df, sequence_length, columns):
    data = df[columns].values
    num_elements = data.shape[0]

    # -1 and +1 because of Python indexing
    for start, stop in zip(range(0, num_elements-(sequence_length-1)), range(sequence_length, num_elements+1)):
        yield data[start:stop, :]

In [ ]:
# Generates sequences for multiple engines
def gen_data_wrapper(df, sequence_length, columns, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
        
    data_gen = (list(gen_train_data(df[df['id']==id], sequence_length, columns))
               for id in ids)
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

In [ ]:
# Functions to generate sequences for the labals
def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    # -1 because I want to predict the rul of that last row in the sequence, not the next row
    return data_matrix[sequence_length-1:num_elements, :]  

def gen_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
        
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array

In [ ]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]
def gen_test_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
    
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    # keep only last window
    if sequence_length > 31:
      print("Too big window")
    else:
      last_labels = [label[-1] for label in label_gen] 
      



    last_labels = np.concatenate(last_labels).astype(np.float32)
    # return label_array
    return last_labels


In [ ]:
# Functions to generate sequences for the labals
def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    # -1 because I want to predict the rul of that last row in the sequence, not the next row
    return data_matrix[sequence_length-1:num_elements, :]  

def gen_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
        
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array

In [ ]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]
def gen_test_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
    
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    # keep only last window
    if sequence_length > 31:
      print("Too big window")
    else:
      last_labels = [label[-1] for label in label_gen] 
      
    last_labels = np.concatenate(last_labels).astype(np.float32)
    # return label_array
    return last_labels


In [ ]:
sequence_length = 20

gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)


for alice_train_unit, alice_val_unit in gss.split(alice_set['id'].unique(), groups=alice_set['id'].unique()):
    alice_train_unit = alice_set['id'].unique()[alice_train_unit]  # gss returns indexes and index starts at 1
    alice_val_unit = alice_set['id'].unique()[alice_val_unit]

    train_split_array = gen_data_wrapper(alice_set, sequence_length, feature_cols, alice_train_unit)
    train_split_label = gen_label_wrapper(alice_set, sequence_length, ['fail_30'], alice_train_unit)
    
    val_split_array = gen_data_wrapper(alice_set, sequence_length, feature_cols, alice_val_unit)
    val_split_label = gen_label_wrapper(alice_set, sequence_length, ['fail_30'], alice_val_unit)

for bob_train_unit, bob_val_unit in gss.split(bob_set['id'].unique(), groups=bob_set['id'].unique()):
    bob_train_unit = bob_set['id'].unique()[bob_train_unit]  # gss returns indexes and index starts at 1
    bob_val_unit = bob_set['id'].unique()[bob_val_unit]

    train_split_array = gen_data_wrapper(bob_set, sequence_length, feature_cols, bob_train_unit)
    train_split_label = gen_label_wrapper(bob_set, sequence_length, ['fail_30'], bob_train_unit)
    
    val_split_array = gen_data_wrapper(bob_set, sequence_length, feature_cols, bob_val_unit)
    val_split_label = gen_label_wrapper(bob_set, sequence_length, ['fail_30'], bob_val_unit)

# create sequences train, test 
X_alice = gen_data_wrapper(alice_set, sequence_length, feature_cols)
X_bob = gen_data_wrapper(bob_set, sequence_length, feature_cols)

y_alice = gen_label_wrapper(alice_set, sequence_length, ['fail_30'])
y_bob = gen_label_wrapper(bob_set, sequence_length, ['fail_30'])


test_gen = (list(gen_test_data(test_set[test_set['id']==id], sequence_length, feature_cols, -99.))
           for id in test_set['id'].unique())
X_test = np.concatenate(list(test_gen)).astype(np.float32)

y_test = gen_test_label_wrapper(test_set, sequence_length, ['fail_30'])


In [ ]:
y_test = y_test.reshape(100,1)

In [ ]:
y_alice.shape

(8959, 1)

# Federated Dataset

In [ ]:
BATCH_SIZE = 64

In [ ]:
# y_alice = y_alice.astype(np.int32)
# y_bob = y_bob.astype(np.int32)

In [ ]:
train_data, val_data = [], []

temp_dataset = tf.data.Dataset.from_tensor_slices((X_alice, y_alice)).batch(BATCH_SIZE)
train_data.append(temp_dataset)
temp_dataset = tf.data.Dataset.from_tensor_slices((X_bob, y_bob)).batch(BATCH_SIZE)
train_data.append(temp_dataset)

temp_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_data.append(temp_dataset.batch(1))

In [ ]:
train_data

[<BatchDataset element_spec=(TensorSpec(shape=(None, 20, 18), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>,
 <BatchDataset element_spec=(TensorSpec(shape=(None, 20, 18), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>]

In [ ]:
val_data

[<BatchDataset element_spec=(TensorSpec(shape=(None, 20, 18), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>]

# Create Model

In [ ]:
def input_spec():
    return (
        tf.TensorSpec([None, 20, 18], tf.float32),
        tf.TensorSpec([None, 1], tf.float32)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(20, 18)),                                       
        tf.keras.layers.LSTM(32, activation='tanh'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[BinaryAccuracy(), Precision(), Recall()])

# Training

In [ ]:
EPOCHS = 100

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam()
)

state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")

Run 100/100

In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.87662166),
                            ('precision', 0.7327434),
                            ('recall', 0.40064517),
                            ('loss', 0.30962804),
                            ('num_examples', 18731),
                            ('num_batches', 293)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.8806257),
                            ('precision', 0.74),
                            ('recall', 0.42967743),
                            ('loss', 0.3002979),
                            ('num_examples', 18731),
                            ('num_batches', 293)]))]),
 OrderedDict([('broadcast', ()),
      

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.93),
                           ('precision', 0.90909094),
                           ('recall', 0.8),
                           ('loss', 0.18721528),
                           ('num_examples', 100),
                           ('num_batches', 100)]))])

In [ ]:
# state.model